# Moodle Quiz Slot Cleanup

This notebook connects to a PostgreSQL database to clean up quiz slots in Moodle.

## Import Libraries

In [ ]:
import psycopg2
from psycopg2 import Error
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy")

In [ ]:
isTesting = True

## Database Configuration

### Reader User

In [ ]:
# Select database user based on testing mode
if isTesting:
    # Use reader user for testing (read-only access)
    DB_HOST = "localhost"
    DB_PORT = "64452"
    DB_NAME = "moodledb"
    DB_USER = "moodledb_reader_user"
    DB_PASSWORD = "YOUR_PASSWORD_HERE"
    print("Using READER user (read-only mode)")
else:
    # Use owner user for actual operations (write access)
    DB_HOST = "localhost"
    DB_PORT = "64452"
    DB_NAME = "moodledb"
    DB_USER = "moodledb_owner_user"
    DB_PASSWORD = "YOUR_PASSWORD_HERE"
    print("Using OWNER user (write access enabled)")

## Connect to Database

In [ ]:
try:
    # Create connection to PostgreSQL database
    connection = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    
    # Create a cursor object
    cursor = connection.cursor()
    
    # Print PostgreSQL connection properties
    print("PostgreSQL server information:")
    print(connection.get_dsn_parameters(), "\n")
    
    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")
    
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

## Helper Functions

In [ ]:
def get_quiz_slots(quizid):
    query = """
    Select id as slotid, slot, quizid, page
    from public.mdl_quiz_slots
    WHERE quizid = %s
    """

    try:
        quizid = int(quizid)
        df = pd.read_sql_query(query, connection, params=(quizid,))
        print(f"Found {len(df)} quiz slots for quiz ID {quizid}")
        return df
    except Exception as e:
        print(f"Error executing query for quiz ID {quizid}: {e}")
        return None

In [ ]:
def get_question_references_for_quiz(quizid):
    query = """
    SELECT *
    FROM public.mdl_question_references
    WHERE itemid IN (
        SELECT id
        FROM public.mdl_quiz_slots
        WHERE quizid = %s
    )
    """
    
    try:
        quizid = int(quizid)
        df = pd.read_sql_query(query, connection, params=(quizid,))
        return df
    except Exception as e:
        print(f"Error executing query for quiz ID {quizid}: {e}")
        return None

In [ ]:
def get_quiz_section_slot_for_quiz(quizid):
    query = """
    SELECT *
    FROM public.mdl_quiz_sections
    WHERE quizid = %s
    """
    
    try:
        quizid = int(quizid)
        df = pd.read_sql_query(query, connection, params=(quizid,))
        return df
    except Exception as e:
        print(f"Error executing query for quiz ID {quizid}: {e}")
        return None

In [ ]:
def get_typeof_random_question_references_for_quiz(quizid):
    query = """
    SELECT *
    FROM public.mdl_question_set_references
    WHERE itemid IN (
        SELECT id
        FROM public.mdl_quiz_slots
        WHERE quizid = %s
    )
    """

    try:
        quizid = int(quizid)
        df = pd.read_sql_query(query, connection, params=(quizid,))
        return df
    except Exception as e:
        print(f"Error executing query for quiz ID {quizid}: {e}")
        return None

In [ ]:
def update_quiz_section_firstslot(quizid, old_slot_to_new_slot_mapping):
    """
    Update quiz section firstslot values based on the slot number changes.
    
    Parameters:
    -----------
    quizid : int
        The quiz ID to update sections for
    old_slot_to_new_slot_mapping : dict
        Dictionary mapping old slot numbers to new slot numbers
        
    Returns:
    --------
    int
        Number of sections updated
    """
    try:
        # Get all quiz sections for this quiz
        sections = get_quiz_section_slot_for_quiz(quizid)
        
        if sections is None or len(sections) == 0:
            print(f"  No quiz sections found for quiz ID {quizid}")
            return 0
        
        cursor = connection.cursor()
        updates_made = 0
        
        for _, section in sections.iterrows():
            old_firstslot = int(section['firstslot'])
            section_id = int(section['id'])
            
            # Check if this firstslot needs to be updated
            if old_firstslot in old_slot_to_new_slot_mapping:
                new_firstslot = old_slot_to_new_slot_mapping[old_firstslot]
                
                update_query = """
                UPDATE public.mdl_quiz_sections
                SET firstslot = %s
                WHERE id = %s
                """
                if isTesting:
                    print(f"  TESTING MODE: Would update section ID {section_id}: firstslot {old_firstslot}→{new_firstslot}")
                    updates_made += 1
                    continue
                cursor.execute(update_query, (new_firstslot, section_id))
                updates_made += 1
                print(f"  Updated section ID {section_id}: firstslot {old_firstslot}→{new_firstslot}")
        
        connection.commit()
        if updates_made > 0:
            print(f"  Quiz ID {quizid}: Successfully updated {updates_made} quiz sections")
        return updates_made
        
    except Exception as e:
        connection.rollback()
        print(f"  Error updating quiz sections for quiz ID {quizid}: {e}")
        return None

In [ ]:
def delete_all_quiz_sections(quizid):
    """
    Delete all quiz sections for a given quiz ID.
    Use this when all quiz slots have been deleted.
    
    Parameters:
    -----------
    quizid : int
        The quiz ID to delete all sections for
        
    Returns:
    --------
    int
        Number of sections deleted
    """
    try:
        quizid = int(quizid)
        
        query = """
        DELETE FROM public.mdl_quiz_sections
        WHERE quizid = %s
        """
        if isTesting:
            print(f"  TESTING MODE: Would delete all quiz sections for quiz ID {quizid}")
            return 0
        cursor = connection.cursor()
        cursor.execute(query, (quizid,))
        sections_deleted = cursor.rowcount
        connection.commit()
        
        if sections_deleted > 0:
            print(f"  Deleted {sections_deleted} quiz sections for quiz ID {quizid}")
        return sections_deleted
        
    except Exception as e:
        connection.rollback()
        print(f"  Error deleting quiz sections for quiz ID {quizid}: {e}")
        return None

In [ ]:
def delete_quiz_slots(slot_ids): 
    if not slot_ids:
        print("No slot IDs provided")
        return 0
    
    # Convert all slot_ids to Python int
    slot_ids = [int(sid) for sid in slot_ids]
    
    # Create placeholders for the IN clause
    placeholders = ','.join(['%s'] * len(slot_ids))
    
    query = f"""
    DELETE FROM public.mdl_quiz_slots
    WHERE id IN ({placeholders})
    """
    
    try:
        if isTesting:
            print(f"  TESTING MODE: Would delete {len(slot_ids)} quiz slots: {slot_ids}")
            return len(slot_ids)
        cursor = connection.cursor()
        cursor.execute(query, slot_ids)
        rows_deleted = cursor.rowcount
        connection.commit()
        print(f"Successfully deleted {rows_deleted} quiz slots")
        return rows_deleted
    except Exception as e:
        connection.rollback()
        print(f"Error deleting quiz slots: {e}")
        return None

In [ ]:
def fix_slot_and_page_ordering(quizid, slots_df):
    """
    Fix the slot and page ordering by filling gaps in the numbering.
    Updates the database to ensure continuous numbering starting from 1.
    Also updates quiz sections if firstslot values need adjustment.
    
    Parameters:
    -----------
    quizid : int
        The quiz ID to fix
    slots_df : pd.DataFrame
        DataFrame with current slots (must be sorted by slot number)
        
    Returns:
    --------
    int
        Number of rows updated
    """
    try:
        cursor = connection.cursor()
        updates_made = 0
        old_slot_to_new_slot_mapping = {}
        
        # Sort by slot to maintain order
        slots_df = slots_df.sort_values('slot').reset_index(drop=True)
        
        # Update each slot to have continuous numbering
        for new_position, (_, row) in enumerate(slots_df.iterrows(), start=1):
            old_slot = int(row['slot'])
            old_page = int(row['page'])
            slot_id = int(row['slotid'])
            
            # Track mapping of old to new slot numbers
            old_slot_to_new_slot_mapping[old_slot] = new_position
            
            # Only update if the slot or page number needs to change
            if old_slot != new_position or old_page != new_position:
                if isTesting:
                    print(f"  TESTING MODE: Would update slot ID {slot_id}: slot {old_slot}→{new_position}, page {old_page}→{new_position}")
                    updates_made += 1
                else:
                    update_query = """
                    UPDATE public.mdl_quiz_slots
                    SET slot = %s, page = %s
                    WHERE id = %s
                    """
                    cursor.execute(update_query, (new_position, new_position, slot_id))
                    updates_made += 1
                    print(f"  Updated slot ID {slot_id}: slot {old_slot}→{new_position}, page {old_page}→{new_position}")
        
        if not isTesting:
            connection.commit()
        print(f"Quiz ID {quizid}: {'Would update' if isTesting else 'Successfully updated'} {updates_made} slots")
        
        # Update quiz sections if there are any slot number changes
        if old_slot_to_new_slot_mapping:
            section_updates = update_quiz_section_firstslot(quizid, old_slot_to_new_slot_mapping)
            if section_updates and section_updates > 0:
                updates_made += section_updates
        
        return updates_made
        
    except Exception as e:
        connection.rollback()
        print(f"Error fixing ordering for quiz ID {quizid}: {e}")
        return None

## Query Quiz Slots Data

In [ ]:
# Define the SQL query
query = """
WITH questionReferences AS (
    SELECT id, usingcontextid
    FROM mdl_question_set_references
), allData as (
SELECT DISTINCT
       qs.quizid,
       q.name,
       q.course,
       q.timecreated,
       qr.id AS question_ref_id,
       c.timecreated as courseTimeCreated,
       c.timemodified as courseTimeModified,
       c.startdate as courseStartDate
FROM mdl_quiz_slots qs
JOIN mdl_quiz q ON q.id = qs.quizid
LEFT JOIN mdl_question_references qr ON qr.itemid = qs.id
LEFT JOIN mdl_course_modules cm ON cm.instance = q.id AND cm.course = q.course
LEFT JOIN mdl_context ctx ON ctx.instanceid = cm.id AND ctx.contextlevel = 70
Left Join mdL_course as c on q.course = c.id
WHERE qr.id IS NULL
  AND ctx.id NOT IN (SELECT usingcontextid FROM questionReferences)
), improvedData as (
SElect to_timestamp(q.courseTimeCreated) as courseTimeCreatedDate, 
to_timestamp(q.courseTimeModified) as  courseTimeModifiedDate, 
to_timestamp(q.courseStartDate)as courseStartDate , * 
from allData as q
)

Select *
from improvedData as i
"""

# Execute query and load into pandas DataFrame
df = pd.read_sql_query(query, connection)

# Display basic information about the DataFrame
print(f"Query returned {len(df)} rows and {len(df.columns)} columns")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head(20)

In [ ]:
# Analyze the current df
print("=== Data Analysis ===\n")

# Count unique course IDs
unique_courses = df['course'].nunique()
print(f"Unique Course IDs: {unique_courses}")

# Count unique quiz IDs
unique_quizzes = df['quizid'].nunique()
print(f"Unique Quiz IDs: {unique_quizzes}")

# Total rows in the dataframe
print(f"Total rows: {len(df)}\n")

# Group by modified year
print("=== Data grouped by Course Modified Year ===")
df['modified_year'] = df['coursetimemodifieddate'].dt.year
year_grouped = df.groupby('modified_year').agg({
    'course': 'nunique',
    'quizid': 'nunique'
}).rename(columns={'course': 'unique_courses', 'quizid': 'unique_quizzes'})

print(year_grouped)

# Also show course created year distribution
print("\n=== Data grouped by Course Created Year ===")
df['created_year'] = df['coursetimecreateddate'].dt.year
year_created_grouped = df.groupby('created_year').agg({
    'course': 'nunique',
    'quizid': 'nunique'
}).rename(columns={'course': 'unique_courses', 'quizid': 'unique_quizzes'})

print(year_created_grouped)

In [ ]:
# Count entries per modified year
year_counts = df['modified_year'].value_counts().sort_index()

# Create the bar chart
plt.figure(figsize=(10, 6))
plt.bar(year_counts.index, year_counts.values, color='steelblue', edgecolor='black')
plt.xlabel('Modified Year', fontsize=12)
plt.ylabel('Number of Entries', fontsize=12)
plt.title('Quiz Slots Count by Course Modified Year', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"\nTotal entries: {year_counts.sum()}")
print(f"Years covered: {year_counts.index.min()} - {year_counts.index.max()}")

In [ ]:
# For Testing, test specific quizIds to simplify debugging
if isTesting:
    # filter dataframe for quizid 4666 or quizid 4667
    #df = df[(df['quizid'] == 10936) | (df['quizid'] == 4667)]
    df = df[(df['quizid'] == 8084)]
    df.head()
else:
    df.head(10)

In [ ]:
allQuizSlots = {}

for quizid in df['quizid'].unique():
    quiz_slots_df = get_quiz_slots(quizid)
    if quiz_slots_df is not None:
        allQuizSlots[quizid] = quiz_slots_df
    else:
        print(f"No data for Quiz ID {quizid}")

In [ ]:
# show all quiz ids with 0 slots
print(f"\nQuiz IDs with 0 slots:")
for quizid in df['quizid'].unique():
    if quizid not in allQuizSlots or len(allQuizSlots[quizid]) == 0:
        print(quizid)

In [ ]:
# extract all quiz with 1 slot
quizzes_with_one_slot = {quizid: slots for quizid, slots in allQuizSlots.items() if len(slots) == 1}
print(f"\nNumber of quizzes with exactly one slot: {len(quizzes_with_one_slot)}")

# filter df to only include quizzes with more than one slot
quizzes_with_more_than_one_slot = {quizid: slots for quizid, slots in allQuizSlots.items() if len(slots) > 1}
print(f"\nNumber of quizzes with more than one slot: {len(quizzes_with_more_than_one_slot)}")

### Find all slot ids without question references

In [ ]:
# Find all slots without question references for quizzes with more than one slot
all_slots_to_delete = {}

for quizid, slots in allQuizSlots.items():
    print('--------------------------------')
    question_refs = get_question_references_for_quiz(quizid)
    question_refs_random = get_typeof_random_question_references_for_quiz(quizid)
    
    print(f"Found {len(question_refs)} question references for quiz ID {quizid}")
    print(f"Found {len(question_refs_random)} random question references for quiz ID {quizid}")
    
    # Combine both reference types - concatenate the DataFrames
    all_question_refs = pd.concat([question_refs, question_refs_random], ignore_index=True)
    print(f"Total combined references: {len(all_question_refs)}")
    
    # find all slots without question references (checking against combined references)
    slots_without_question_refs = []
    for _, slot in slots.iterrows():
        if all_question_refs is not None and len(all_question_refs) > 0:
            if slot['slotid'] not in all_question_refs['itemid'].values:
                slots_without_question_refs.append(slot)
        else:
            # If no references at all, all slots are without references
            slots_without_question_refs.append(slot)
    
    if slots_without_question_refs:
        all_slots_to_delete[quizid] = slots_without_question_refs
        print(f"Quiz ID {quizid} has {len(slots)} slots, of which {len(slots_without_question_refs)} have no question references.\n")

print(f"\n\nTotal quizzes with slots to delete: {len(all_slots_to_delete)}")

### Delete the Quiz Slot Ids without references

In [ ]:
# Delete slots for all quizzes
total_deleted = 0
total_sections_deleted = 0

if all_slots_to_delete:
    for quizid, slots_without_question_refs in all_slots_to_delete.items():
        # Convert list of Series to list of slot IDs (as Python int)
        slots_to_delete = [int(slot['slotid']) for slot in slots_without_question_refs]
        print(f"\nQuiz ID {quizid} - Slot IDs to delete: {slots_to_delete}")
        
        # Check if we're deleting ALL slots for this quiz
        total_slots_for_quiz = len(allQuizSlots[quizid])
        deleting_all_slots = (len(slots_to_delete) == total_slots_for_quiz)
        
        deleted_count = delete_quiz_slots(slots_to_delete)
        if deleted_count is not None:
            total_deleted += deleted_count
            
            # If all slots were deleted, also delete all quiz sections
            if deleting_all_slots:
                print(f"  All slots deleted for quiz ID {quizid}, deleting quiz sections...")
                sections_deleted = delete_all_quiz_sections(quizid)
                if sections_deleted is not None:
                    total_sections_deleted += sections_deleted
    
    print(f"\n\n=== Deletion Summary ===")
    print(f"Total quizzes processed: {len(all_slots_to_delete)}")
    print(f"Total slots deleted: {total_deleted}")
    print(f"Total quiz sections deleted: {total_sections_deleted}")
else:
    print("No slots to delete.")

## Fix Slot and Page Ordering After Deletions

In [ ]:
# Re-fetch quiz slots after deletions and check for gaps in ordering
quizzes_needing_reordering = {}

for quizid in quizzes_with_more_than_one_slot.keys():
    # Get current slots after deletion
    current_slots = get_quiz_slots(quizid)
    
    if current_slots is not None and len(current_slots) > 0:
        # Sort by slot number
        current_slots = current_slots.sort_values('slot')
        
        # Check if there are gaps in slot numbering
        expected_slots = list(range(1, len(current_slots) + 1))
        actual_slots = current_slots['slot'].tolist()
        
        # Check if there are gaps in page numbering
        expected_pages = list(range(1, len(current_slots) + 1))
        actual_pages = current_slots['page'].tolist()
        
        if actual_slots != expected_slots or actual_pages != expected_pages:
            print(f"\nQuiz ID {quizid} needs reordering:")
            print(f"  Current slots: {actual_slots}")
            print(f"  Expected slots: {expected_slots}")
            print(f"  Current pages: {actual_pages}")
            print(f"  Expected pages: {expected_pages}")
            quizzes_needing_reordering[quizid] = current_slots

print(f"\n\nTotal quizzes needing reordering: {len(quizzes_needing_reordering)}")

In [ ]:
# Apply the fix to all quizzes that need reordering
total_updates = 0
for quizid, slots_df in quizzes_needing_reordering.items():
    print(f"\nFixing Quiz ID {quizid}...")
    updates = fix_slot_and_page_ordering(quizid, slots_df)
    if updates is not None:
        total_updates += updates

print(f"\n\n=== Summary ===")
print(f"Total quizzes fixed: {len(quizzes_needing_reordering)}")
print(f"Total slot updates: {total_updates}")